In [209]:
import pandas as pd
import numpy as np
%config IPCompleter.greedy=True


In [210]:
df = pd.read_csv('MyFoodData.csv')
df.columns = ['ID', 'Name', 'Group', 'Carbohydrate', 'Ranking']
min_carb_intake = 10
max_carb_intake = 200
target_avg_rank = 8
fridge_items = 4
df = df[df.Ranking > 1] # removing items with rank less than 1 because in worst case they wouldn't meet avg
df = df[df.Group != 'Fats and Oils'] # this is an ingredient of foods, no one eats this by itself
# sanity check that all inputs are valid

In [212]:
fridge = df.sample(n=fridge_items, replace=False, ignore_index=True)
fridge = fridge.sort_values(by=['Ranking'], ascending=False)
# menu = pd.DataFrame(columns = ['ID', 'Name', 'Group', 'Carbohydrate', 'Ranking']) #this may not be needed
# average_rank = fridge['Ranking'].mean()
# total_carbs = fridge['Carbohydrate'].sum()
# item_counts = menu['ID'].count()

#initializing a list of beverages 
drinks = fridge[(fridge['Group'] == 'Beverages')]
#removing drinks from the fridge
fridge = fridge[(fridge['Group'] != 'Beverages')]
#you can reduce the search space as you go because there are cases that will not meet the conditions
drinks

,ID,Name,Group,Carbohydrate,Ranking
3,171886,Cocoa Mix No Sugar Added Powder,Beverages,72,5


In [218]:
fridge_n = fridge[['ID','Carbohydrate','Ranking']].to_numpy()
drinks_n = drinks[['ID','Carbohydrate','Ranking']].to_numpy()
menu = np.zeros((4,3), dtype='int64')
# menu = np.vstack((fridge_n,fridge_n[0,:]))
fridge

,ID,Name,Group,Carbohydrate,Ranking
1,168929,Cornmeal Degermed Unenriched Yellow,Grains and Pasta,79,8
2,788122,Cauliflower Cooked From Fresh Fat Added In Coo...,Vegetables,4,3
0,173841,Veal Sirloin Separable Lean Only Cooked Braised,Meats,0,2


In [217]:
for idx in range(drinks_n.shape[0]):
    if drinks_n.size == 0:
        print('there is no valid menu because there are no drinks')
        break
    else:
        if drinks_n[idx,1] < max_carb_intake: # checking if carbs in item are at least smaller than max
            menu[0] = drinks_n[idx]
            #print(menu)
            for idx2 in range(fridge_n.shape[0]):
                if fridge_n[idx2,0] in menu:
                    break
                else:
                    if round((menu[:,2].sum() + fridge_n[idx2,2])/(menu.shape[0]+1)) >= 8:
                        print('yes')
        

In [151]:
if drinks_n[ix,0] in drinks_n:
    print('yes')
else :
    print('no')
    
if round((menu[:,2].sum() + drinks_n[idx,2])/(menu.shape[0]+1)) >= 8:

yes


In [170]:
fridge_n

array([[170440,     20,     10],
       [788093,      5,      7],
       [783111,     14,      2]], dtype=int64)